In [ ]:
using Revise, LazySets, Test, BenchmarkTools 

In [ ]:
Z = rand(Zonotope, num_generators=30)

In [ ]:
order(Z)

In [ ]:
X = reduce_order(Z, 3)

In [ ]:
order(X)

In [ ]:
X.generators

In [ ]:
Z.generators

---

In [1]:
using Revise, ReachabilityAnalysis, Test, BenchmarkTools
const RA = ReachabilityAnalysis

┌ Info: Precompiling ReachabilityAnalysis [1e97bd63-91d1-579d-8e8d-501d2b57c93f]
└ @ Base loading.jl:1260


ReachabilityAnalysis

In [13]:
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/linear/motor.jl")

prob, tspan = motor_homog();
A = state_matrix(prob)
X0 = initial_state(prob)
probIM = @ivp(x' = IntervalMatrix(A) *x, x(0) ∈ X0);

In [17]:
sol = solve(probIM, (0.0, 1.0), ASB07(δ=1e-3, max_order=3));

In [ ]:


@inferred solve(prob, (0.0, 1.0), GLGM06(δ=1e-3, max_order=3));

In [ ]:
versioninfo()

In [ ]:
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/linear/exponential1D.jl")
prob, tspan = exponential_1d()
@inferred solve(prob, (0.0, 1.0), INT(δ=1e-3));

In [ ]:
@btime solve($prob, (0.0, 1.0), INT(δ=0.01));

In [ ]:
@inferred solve(prob, (0.0, 1.0));

In [ ]:
@btime solve($prob, (0.0, 1.0));

In [ ]:
sol = solve(prob, (0.0, 1.0))
sol.alg

In [ ]:
@btime solve($prob, (0.0, 1.0), GLGM06(δ=1e-3, max_order=3));

In [ ]:
using Test

@inferred solve(prob, (0.0, 0.1), GLGM06(δ=1e-3, max_order=3));

In [ ]:
reduce_order(rand(Zonotope), num_)

In [ ]:
ivp_norm = RA._normalize(probIM);
ivp_discr = RA.discretize(ivp_norm, sol.alg.δ, sol.alg.approx_model);

Φ = state_matrix(ivp_discr)
Ω0 = initial_state(ivp_discr)
X = stateset(ivp_discr);

In [ ]:
# split the interval matrix into center and radius
Φc, Φs = RA._split(Φ);

In [ ]:
δ = sol.alg.δ
N = Float64
Δt = zero(N) .. δ
F1 = ReachSet(Ω0, Δt);
Zk = set(F1);

In [ ]:
Rₖ = RA._overapproximate_interval_linear_map(Φc, Φs, Zk.center, Zk.generators)

In [ ]:
size(Ω0.generators)

In [ ]:
size(Rₖ.generators)

In [ ]:
sol.alg.max_order

In [ ]:
Rkred = RA._reduce_order(Rₖ, 3)

In [ ]:
size(Ω0.generators)
order(Ω0) * 1.0

In [ ]:
order(Rₖ) * 1.0

In [ ]:
size(Rₖ.generators)

In [ ]:
size(Rkred.generators)
order(Rkred) * 1.0

In [ ]:
Rkred = reduce_order(Rₖ, 3);

In [ ]:
size(Rkred.generators)
order(Rkred) * 1.0

In [ ]:
    
    @inbounds F[1] = ReachSet(Ω0, Δt)


    k = 1
    @inbounds while k <= NSTEPS - 1
        Zk = set(F[k])
        ck = Zk.center
        Gk = Zk.generators
        Rₖ = _overapproximate_interval_linear_map(Φc, Φs, ck, Gk)
        Rₖ = _reduce_order(Rₖ, max_order)
        println("k = $k, order = $(order(Rₖ))")
        Δt += δ
        k += 1
        F[k] = ReachSet(Rₖ, Δt)
    end

In [ ]:

# true <=> there is no input, i.e. the system is of the form x' = Ax, x ∈ X
got_homogeneous = !hasinput(ivp_discr)

# this algorithm requires Ω0 to be a zonotope
Ω0 = _convert_or_overapproximate(Zonotope, Ω0)
Ω0 = _reduce_order(Ω0, max_order)

In [ ]:
Φc, Φs = RA._split(Φ)

In [ ]:
order(set(sol[2]))

In [ ]:
RA.probIM.s

In [ ]:
maximum([order(set(sol[i])) for i in 1:length(sol)])

In [18]:
include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/linear/motor.jl")

prob, tspan = motor_homog();
A = state_matrix(prob)
X0 = initial_state(prob)
probIM = @ivp(x' = IntervalMatrix(A) *x, x(0) ∈ X0);

In [53]:
function reach_homog_GLGM06!(F::Vector{ReachSet{N, Zonotope{N, VN, MN}}},
                             Ω0::Zonotope{N, VN, MN},
                             Φ::AbstractMatrix,
                             NSTEPS::Integer,
                             δ::Float64,
                             max_order::Integer,
                             X::Universe) where {N, VN, MN}

    # initial reach set
    Δt = zero(N) .. δ
    @inbounds F[1] = ReachSet(Ω0, Δt)

    k = 2
    @inbounds while k <= NSTEPS
        Rₖ = RA._linear_map(Φ, set(F[k-1]))
        Δt += δ
        F[k] = ReachSet(Rₖ, Δt)
        k += 1
    end
    return F
end

function reach_homog_GLGM06_v2!(F::Vector{ReachSet{N, Zonotope{N, VN, MN}}},
                             Ω0::Zonotope{N, VN, MN},
                             Φ::AbstractMatrix,
                             NSTEPS::Integer,
                             δ::Float64,
                             max_order::Integer,
                             X::Universe) where {N, VN, MN}
    
    # initial reach set
    Δt = zero(N) .. δ
    @inbounds F[1] = ReachSet(Ω0, Δt)

    n, p = size(Ω0.generators)

    # preallocate output
    Zout = Vector{Zonotope{N, Vector{N}, Matrix{N}}}(undef, NSTEPS)
    Zout[1] = Ω0
    @inbounds for i in 2:NSTEPS
        c = Vector{N}(undef, n)
        G = Matrix{N}(undef, n, p)
        Zout[i] = Zonotope(c, G)
    end

    k = 2
    @inbounds while k <= NSTEPS
        RA._linear_map!(Zout[k], Φ, Zout[k-1])
        Δt += δ
        F[k] = ReachSet(Zout[k], Δt)
        k += 1
    end
    return F
end

reach_homog_GLGM06_v2! (generic function with 1 method)

In [ ]:
post(GLGM06(δ=0.01), prob, tspan)

In [47]:
using Parameters

function post(alg::GLGM06, ivp::IVP{<:AbstractContinuousSystem}, tspan)

    @unpack δ, approx_model, max_order = alg

    # get time horizon from the time span imposing that it is of the form (0, T)
    T = RA._get_T(tspan, check_zero=true, check_positive=true)
    NSTEPS = ceil(Int, T / δ)

    # normalize system to canonical form
    ivp_norm = RA._normalize(ivp)

    # discretize system
    ivp_discr = RA.discretize(ivp_norm, δ, approx_model)
    Φ = state_matrix(ivp_discr)
    Ω0 = initial_state(ivp_discr)
    X = stateset(ivp_discr)

    # this algorithm requires Ω0 to be a zonotope
    Ω0 = RA._convert_or_overapproximate(Zonotope, Ω0)
    Ω0 = RA._reduce_order_GIR05(Ω0, max_order)

    # reconvert the set of initial states and state matrix, if needed
    static = alg.static
    Ω0 = RA._reconvert(Ω0, Val(static))
    Φ = RA._reconvert(Φ, Val(static))

    # preallocate output flowpipe
    N = eltype(Ω0)
    ZT = typeof(Ω0)
    F = Vector{ReachSet{N, ZT}}(undef, NSTEPS)

    reach_homog_GLGM06!(F, Ω0, Φ, NSTEPS, δ, max_order, X)
    return Flowpipe(F)
end

function post2(alg::GLGM06, ivp::IVP{<:AbstractContinuousSystem}, tspan)

    @unpack δ, approx_model, max_order = alg

    # get time horizon from the time span imposing that it is of the form (0, T)
    T = RA._get_T(tspan, check_zero=true, check_positive=true)
    NSTEPS = ceil(Int, T / δ)

    # normalize system to canonical form
    ivp_norm = RA._normalize(ivp)

    # discretize system
    ivp_discr = RA.discretize(ivp_norm, δ, approx_model)
    Φ = state_matrix(ivp_discr)
    Ω0 = initial_state(ivp_discr)
    X = stateset(ivp_discr)

    # this algorithm requires Ω0 to be a zonotope
    Ω0 = RA._convert_or_overapproximate(Zonotope, Ω0)
    Ω0 = RA._reduce_order_GIR05(Ω0, max_order)

    # reconvert the set of initial states and state matrix, if needed
    static = alg.static
    Ω0 = RA._reconvert(Ω0, Val(static))
    Φ = RA._reconvert(Φ, Val(static))

    # preallocate output flowpipe
    N = eltype(Ω0)
    ZT = typeof(Ω0)
    F = Vector{ReachSet{N, ZT}}(undef, NSTEPS)

    reach_homog_GLGM06_v2!(F, Ω0, Φ, NSTEPS, δ, max_order, X)
    return Flowpipe(F)
end

post2 (generic function with 1 method)

In [42]:
alg = GLGM06(δ=1e-3)
tspan = RA.TimeInterval(0.0, 20.0)

[0, 20]

In [43]:
@inferred post(alg, prob, tspan)

ErrorException: return type Flowpipe{Float64,ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}} does not match inferred return type Flowpipe

In [46]:
@benchmark post($alg, $prob, $tspan) # using _linear_map

BenchmarkTools.Trial: 
  memory estimate:  16.20 MiB
  allocs estimate:  80291
  --------------
  minimum time:     50.192 ms (0.00% GC)
  median time:      65.423 ms (0.00% GC)
  mean time:        70.957 ms (4.80% GC)
  maximum time:     149.083 ms (14.80% GC)
  --------------
  samples:          71
  evals/sample:     1

In [55]:
post2(alg, prob, tspan)[end]

ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}(Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.314e-321, -2.9e-322, -1.5e-323, -2.5e-323, 7.95e-322, -1.023e-321, -1.0e-323, -1.5e-323], [1.8072189437198342e-31 0.0 … 0.0 0.0; -0.0 3.0e-323 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 3.1433903644581805e-7]), [19.9989, 20.0001])

In [56]:
post(alg, prob, tspan)[end]

ReachSet{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}(Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.314e-321, -2.9e-322, -1.5e-323, -2.5e-323, 7.95e-322, -1.023e-321, -1.0e-323, -1.5e-323], [2.47e-322 -1.84e-321 … 0.0 0.0; 2.01e-321 1.83e-322 … 0.0 0.0; … ; 0.0 0.0 … 1.0e-323 1.0e-323; 0.0 0.0 … 2.0e-323 2.0e-323]), [19.9989, 20.0001])

In [57]:
@benchmark post2($alg, $prob, $tspan) # using inplace _linear_map!

BenchmarkTools.Trial: 
  memory estimate:  19.10 MiB
  allocs estimate:  100292
  --------------
  minimum time:     21.078 ms (0.00% GC)
  median time:      27.629 ms (0.00% GC)
  mean time:        31.960 ms (14.02% GC)
  maximum time:     81.638 ms (30.93% GC)
  --------------
  samples:          157
  evals/sample:     1

In [60]:
alg = GLGM06(δ=1e-3, max_order=1)

GLGM06{Float64,Forward}
  δ: Float64 0.001
  approx_model: Forward
  max_order: Int64 1
  static: Bool false


In [61]:
@btime post2($alg, $prob, $tspan);

  19.632 ms (100292 allocations: 19.10 MiB)


In [62]:
@btime post($alg, $prob, $tspan);

  33.054 ms (80291 allocations: 16.20 MiB)
